In [ ]:

 
import json
import os 
import urllib.request
import datetime
import boto3
from collections import Counter

from collections import Counter
import string
 
import numpy as np
from nltk.tokenize import word_tokenize
import simplejson as json
 
from nlp import build_model
 
def process_str(s):
     s = s.translate(None, string.punctuation)
    return word_tokenize(s.lower())
 
def read_dataset(sentences, labels):
     dataset = []
    with open(sentences) as sentences, open(labels) as labels:
        import ipdb; ipdb.set_trace()
        for s, l in zip(sentences, labels):
            try:
                words = process_str(s)
                dataset.append( (int(l), set(words)) )
            except ValueError:
                pass
    return dataset
 
def get_most_commons(dataset, skip=10, total=4000):
    my_list = []
    for item in dataset:
        my_list += list(item[1])

           counter = Counter(my_list)

               temp = counter.most_common(total+skip)[skip:]
    words = [item[0] for item in temp]
    return words
def generate_vectors(dataset, common_words, ternary, bias_term):
    d = {}
    for i in range(len(common_words)):
        d[common_words[i]] = i
         vectors = []
    for item in dataset:
        vector = [0] * len(common_words)
        for word in item[1]:
             if word in d:
                vector[d[word]] += 1

            if bias_term:
            vector.append(1.0)
 
         vectors.append( (item[0], np.array(vector)) )
 
   return vectors

def random_sample(data, proportion):

    count = int(len(data) * proportion)
    np.random.shuffle(data)
    return data[:count]


def evaluate(preds, golds):
    tp, pp, cp = 0.0, 0.0, 0.0
    for pred, gold in zip(preds, golds):
        if pred == 1:
            pp += 1
        if gold == 1:
            cp += 1
        if pred == 1 and gold == 1:
            tp += 1
    precision = tp / pp
    recall = tp / cp
    try:
        f1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        return (precision, recall, 0.0)
    return (precision, recall, f1)
 
 
 def build_dataset(sentences, labels, nb_words=500):
        with open(sentences) as f:
            data = f.readlines()
        with open(labels) as g:
            labels = [int(label) for label in g.read()[:-1].split("\n")]
 
    ternary = True
 
     train_data = [set(process_str(s)) for s in data]
          train_data = [(label, data) for label, data in zip(labels, train_data)]
 
     common_words = get_most_commons(train_data, total=nb_words)
 
     train_vectors = generate_vectors(train_data, common_words, ternary, False)
 
     X = np.array([e[1] for e in train_vectors])
     Y = np.array([e[0] for e in train_vectors]).reshape(-1, 1)
 
    data = np.append(X, Y, axis=1)
     np.random.shuffle(data)
 
     X = data[:, :-1]
     Y = data[:, -1].reshape(-1, 1)
 
     return X, Y, common_words
 
 
 if __name__ == '__main__':
 
    batch_size = 128
    nb_classes = 2
     nb_epoch = 12
         nb_words = 4000
 
     X, Y, common_words = build_dataset('sentences.txt', 'labels.txt', nb_words=nb_words)
 
     model = build_model(nb_words)
     model.fit(X[:7000], Y[:7000], nb_epoch=nb_epoch, batch_size=batch_size)
         score = model.evaluate(X[7000:], Y[7000:])
             print('Loss: ', score[0], ' Accuracy: ', score[1])
 
     with open('parsed_chat.json') as f:
         chat_file = json.load(f)
 
     messages = [message['message'] for message in chat_file['threads'][0]['messages'] if message['sender'] == '']
     message = [(0, ' '.join(messages))]
    import ipdb; ipdb.set_trace()
     message_vector = generate_vectors(message, common_words, False, False)
         X = np.array([message_vector[0][1]])
             print(model.predict(X))

class StreamHandler():
    def __init__(self):
        self.fb_access_token = None
        self.fb_access_token_secret = None
        self.fb_consumer_key = None
        self.fb_consumer_secret = None
        self.fb_filters_list = None
        self.fb_languages_list = None

    def FetchLastPictures(self):
        stream_callback = FetchedPictureHandler()
        auth = fb.OAuthHandler(self.fb_consumer_key, self.fb_consumer_secret)
        auth.set_access_token(self.fb_access_token, self.fb_access_token_secret)  
        stream = fb.Stream(auth, stream_callback,timeout=120)
        stream.filter(languages=self.fb_languages_list ,track=self.fb_filters_list)


class FetchedPictureHandler(fb.StreamListener):
    def on_data(self, data):
        global NUMBER_OF_FETCHED_PICTURES
        if NUMBER_OF_FETCHED_PICTURES < NUMBER_OF_PICTURE_TO_FECTH:
            try:
                decoded = json.loads(data)
                if 'extended_entities' in decoded:
                    for media in decoded['extended_entities']['media']:
                        pic_url = media['media_url_https']
                        pic_name = '{}{}_{}'.format(IMG_OUTPUT_LOCATION, datetime.datetime.now().strftime("%Y%m%d-%H%M%S%f"), os.path.basename(pic_url))
                        urllib.request.urlretrieve(pic_url, pic_name)
            
                        #check if is there a face within the picture
                        p = open(pic_name, 'rb')
                        reko_response = rekognition_client_connection.detect_faces(Image={
                                                                            'Bytes':bytearray(p.read())
                                                                        },
                                                                            Attributes=[
                                                                            'ALL'
                                                                        ])
                        p.close()
                        if 'FaceDetails' in reko_response and len(reko_response['FaceDetails'])>0:
                            for detail in reko_response['FaceDetails']:
                                #store details
                                if 'Smile' in detail:
                                    SMILES.append(detail['Smile']['Value'])
                                if 'Gender' in detail:
                                    GENDERS.append(detail['Gender']['Value'])

                                if 'Emotions' in detail:
                                    for e in detail['Emotions']:
                                        if e['Type'] not in EMOTIONS:
                                            EMOTIONS[e['Type']] = []
                                        EMOTIONS[e['Type']].append(e['Confidence'])

                            NUMBER_OF_FETCHED_PICTURES = NUMBER_OF_FETCHED_PICTURES + 1
                        else:
                            os.remove(pic_name)
            except Exception as e:
                print(e)
                pass
            return True
        else:
            return False
def get_text_sentiment(text):

    alchemy_api_key = os.environ.get("ALCHEMY_API_KEY")
    alchemy_language = AlchemyLanguage(api_key=alchemy_api_key)
    result = alchemy_language.sentiment(text=text)
    if result['docSentiment']['type'] == 'neutral':
        return 'netural', 0
        return result['docSentiment']['type'], result['docSentiment']['score']
            def on_error(self, status):
        # TODO maybe add some log?
            print(status)

AWS_ACCESS_KEY = ''
AWS_SECRET_ACCESS_KEY =''
rekognition_client_connection = boto3.client(
    'rekognition',
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    region_name = 'eu-west-1'
)


NUMBER_OF_FETCHED_PICTURES = 0
NUMBER_OF_PICTURE_TO_FECTH = 25 # number of iamges to download
IMG_OUTPUT_LOCATION = 'img_out/' # output folder

SMILES = []
GENDERS = []
EMOTIONS = {}

if __name__ == '__main__':
    try:
        fb_handler = StreamHandler()
        fb_handler.fb_access_token = ""
        fb_handler.fb_access_token_secret = ""
        fb_handler.fb_consumer_key = ""
        fb_handler.fb_consumer_secret = ""
        fb_handler.fb_filters_list = ['#christmas', '#family', '#friends', '#love', '#faces'] # set of filters
        fb_handler.fb_languages_list = ['en']
        fb_handler.FetchLastPictures()

		# print the output
        print("SMILING:")
        for i,j in zip(Counter(SMILES).keys(), Counter(SMILES).values()):
            print('{} : {}'.format(i, j))
        
        print("GENDER:")
        for i,j in zip(Counter(GENDERS).keys(), Counter(GENDERS).values()):
            print('{} : {}'.format(i, j))
        
        print("EMOTIONS:")
        for key, value in EMOTIONS.items():
            print('{} : {}'.format(key, sum(value) / float(len(value))))

    except Exception as e:
        print(e)
